# โค้ดแปลงเวลา UTC2Local สร้างลิสต์ไฟล์สำหรับนำไปประมวลผลแบบ batch

In [1]:
'''
2024/01/09 
พัฒนาโค้ดโดย รศ.ดร.นัฐพล มหาวิค 
email:nattaponm@nu.ac.th 
ยูทูป:https://www.youtube.com/@Nattapon_Mahavik/playlists 
หนังสือ: เรดาร์ตรวจอากาศทางอุตุนิยมวิทยา https://www.chulabook.com/education/144567
Github: https://github.com/nattaponm
Scopus: https://www.scopus.com/authid/detail.uri?authorId=55583236600
'''

'\n2024/01/09 \nพัฒนาโค้ดโดย รศ.ดร.นัฐพล มหาวิค \nemail:nattaponm@nu.ac.th \nยูทูป:https://www.youtube.com/@Nattapon_Mahavik/playlists \nหนังสือ: เรดาร์ตรวจอากาศทางอุตุนิยมวิทยา https://www.chulabook.com/education/144567\nGithub: https://github.com/nattaponm\nScopus: https://www.scopus.com/authid/detail.uri?authorId=55583236600\n'

In [2]:
import numpy as np
import matplotlib.pyplot as pl
import pyart


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


In [3]:
radar = pyart.io.read('../1data/0radar/1radar_tmd/PHS240@201807201000.uf.bz2') #พิษณุโลก
print(radar.fields.keys())

dict_keys(['reflectivity', 'velocity', 'spectrum_width', 'corrected_reflectivity', 'corrected_differential_reflectivity', 'differential_phase', 'cross_correlation_ratio'])


# ตรวจสอบเวลาของไฟล์

In [4]:
time_rad=radar.time['units'][14:]
print(time_rad)

2018-07-20T10:00:03Z


## ตรวจสอบเวลา local / UTC
* https://dev.to/behainguyen/python-local-date-time-and-utc-date-time-4cl7#:~:text=astimezone(tz%3DNone).&text=We%20can%20see%20that%20datetime,the%20original%20UTC%20offset%20value.

In [5]:
import time

local_time = time.localtime()

print("Time Zone: ", time.tzname)
print("Time Zone: ", time.strftime("%Z", local_time))
print("Date and Time Zone: ", time.strftime("%Y-%m-%d %H:%M:%S %Z", local_time) )
print("UTC Offset: ", time.strftime("%z", local_time))

Time Zone:  ('SE Asia Standard Time', 'SE Asia Daylight Time')
Time Zone:  SE Asia Standard Time
Date and Time Zone:  2024-01-09 19:03:40 SE Asia Standard Time
UTC Offset:  +0700


## การเปลี่ยน utc ไปเป็น local time
* โค้ด https://stackoverflow.com/questions/4770297/convert-utc-datetime-string-to-local-datetime

In [6]:
# แปลงเวลา utc2local https://stackoverflow.com/questions/4770297/convert-utc-datetime-string-to-local-datetime
from datetime import datetime
from dateutil import tz

#Auto-detect zones:
from_zone = tz.tzutc()
to_zone = tz.tzlocal()

utc = datetime.strptime(time_rad, "%Y-%m-%dT%H:%M:%SZ")
utc = utc.replace(tzinfo=from_zone)

# Convert time zone
lst = utc.astimezone(to_zone)
print('utc: ', utc)
print('lst: ', lst)

utc:  2018-07-20 10:00:03+00:00
lst:  2018-07-20 17:00:03+07:00


In [7]:
time_rad_lst = lst.strftime('%Y%m%d%H')
print(time_rad_lst)

2018072017


## การทำลิสต์ชื่อไฟล์ราย 15 นาที ให้เป็นรายชั่วโมง

In [8]:
# ลูปไฟล์ uf ทั้งหมดในโฟลเดอร์ > แปลงเวลา utc2local ราย 15 นาที > แอดเวลา local เข้าไปใน list 
# > แล้วลดเวลา 15 นาที ให้เหลือเวลาเฉพาะ 1 ชั่วโมง


In [9]:
import os
dir_path = '../1data/0radar/1radar_tmd/'

#สร้างลิสต์ชื่อไฟล์ทุกไฟล์ที่มีในโฟลเดอร์นั้น ๆ 
fn_all = []
for file in os.listdir(dir_path):
    if file.endswith('.bz2'):
        fn_all.append(file)
print(fn_all)


['PHS240@201807201000.uf.bz2', 'PHS240@201807201030.uf.bz2', 'PHS240@201807201045.uf.bz2', 'PHS240@201807201100.uf.bz2', 'PHS240@201807201115.uf.bz2', 'PHS240@201807201130.uf.bz2', 'PHS240@201807201145.uf.bz2']


In [10]:
#สร้างลิสต์ชื่อไฟล์รายชั่วโมง
fn_hr=[]
for file in fn_all:
    fn_hr.append(file[7:-9])
    #print(file[3:-7])

fn_hr=list(np.unique(fn_hr)) # ชื่อไฟล์ในแต่ละชั่วโมง
print(fn_hr)

['2018072010', '2018072011']


In [11]:
#เปลี่ยน UTC ไปเป็น Local ชื่อไฟล์รายชั่วโมง
from datetime import datetime
from dateutil import tz


def utc2local_hr(time_rad):
    from_zone = tz.tzutc()
    to_zone = tz.tzlocal()

    utc = datetime.strptime(time_rad, "%Y%m%d%H")
    utc = utc.replace(tzinfo=from_zone)

    # Convert time zone
    lst = utc.astimezone(to_zone)
    print('utc: ', utc)
    print('lst: ', lst)
    return lst

#ลูปชื่อไฟล์รายชั่วโมงในลิสต์ เพื่อจะ get ค่าไฟล์เรดาร์ในชั่วโมงดังกล่าว จะใช้ลิสต์ชื่อไฟล์นี้นี้ในการเซฟเป็นส่วนหนึ่งของชื่อไฟล์
for file in fn_hr:
    print(utc2local_hr(file).strftime('%Y%m%d%H'))

utc:  2018-07-20 10:00:00+00:00
lst:  2018-07-20 17:00:00+07:00
2018072017
utc:  2018-07-20 11:00:00+00:00
lst:  2018-07-20 18:00:00+07:00
2018072018


# วางแผนลูปในการประมวลผลแบบ batch เพื่อสร้าง cappi/ppi แต่ละระดับและมุมยก

In [12]:
for sub_file_hr in fn_hr:
    print('substring file UTC time in hourly: ', sub_file_hr)
    files_15proc = [file for file in fn_all if sub_file_hr in file]
    print('file names  UTC time in 15 min: ', files_15proc)
    print('do convert UTC to Local time, create file names according to features')
    for fn_ in files_15proc:
        print('doing ...',fn_ )        
        print('do processing SNR, Attenuation Correction, Gridding, Cappi/PPI')    
    

substring file UTC time in hourly:  2018072010
file names  UTC time in 15 min:  ['PHS240@201807201000.uf.bz2', 'PHS240@201807201030.uf.bz2', 'PHS240@201807201045.uf.bz2']
do convert UTC to Local time, create file names according to features
doing ... PHS240@201807201000.uf.bz2
do processing SNR, Attenuation Correction, Gridding, Cappi/PPI
doing ... PHS240@201807201030.uf.bz2
do processing SNR, Attenuation Correction, Gridding, Cappi/PPI
doing ... PHS240@201807201045.uf.bz2
do processing SNR, Attenuation Correction, Gridding, Cappi/PPI
substring file UTC time in hourly:  2018072011
file names  UTC time in 15 min:  ['PHS240@201807201100.uf.bz2', 'PHS240@201807201115.uf.bz2', 'PHS240@201807201130.uf.bz2', 'PHS240@201807201145.uf.bz2']
do convert UTC to Local time, create file names according to features
doing ... PHS240@201807201100.uf.bz2
do processing SNR, Attenuation Correction, Gridding, Cappi/PPI
doing ... PHS240@201807201115.uf.bz2
do processing SNR, Attenuation Correction, Gridding

# ให้ทดสอบ

In [13]:
#*... ให้ทำความเข้าใจกับการสร้างลิสต์รายชั่วโมง/ลิสต์15นาที
#สร้างลิสต์ชื่อไฟล์ทุกไฟล์ที่มีในโฟลเดอร์นั้น ๆ 
fn_all = []
for file in os.listdir(dir_path):
    if file.endswith('.bz2'):
        fn_all.append(file)
print(fn_all)

#และ 
#สร้างลิสต์ชื่อไฟล์รายชั่วโมง
fn_hr=[]
for file in fn_all:
    fn_hr.append(file[7:-9])
    #print(file[3:-7])

fn_hr=list(np.unique(fn_hr)) # ชื่อไฟล์ในแต่ละชั่วโมง
print(fn_hr)


['PHS240@201807201000.uf.bz2', 'PHS240@201807201030.uf.bz2', 'PHS240@201807201045.uf.bz2', 'PHS240@201807201100.uf.bz2', 'PHS240@201807201115.uf.bz2', 'PHS240@201807201130.uf.bz2', 'PHS240@201807201145.uf.bz2']
['2018072010', '2018072011']
